In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import hashlib
import time
import shutil

In [3]:
MNM_nb_folder = os.path.join('..', '..', '..', 'side_project', 'network_builder')
sys.path.append(MNM_nb_folder)
python_lib_folder = os.path.join('..', '..', 'pylib')
sys.path.append(python_lib_folder)

In [4]:
from MNM_nb import *
import MNMAPI
from DODE import *

In [5]:
data_folder = os.path.join('..', '..', '..', 'data', 'input_files_2link_fix')

In [6]:
nb = MNM_network_builder()
nb.load_from_folder(data_folder)

MNM_config
MNM_pathtable


In [7]:
nb.link_list

[MNM_dlink, ID: 1, type: PQ, length: 1.0 miles, ffs: 99999.0 mi/h,
 MNM_dlink, ID: 2, type: PQ, length: 1.0 miles, ffs: 99999.0 mi/h,
 MNM_dlink, ID: 3, type: CTM, length: 0.8 miles, ffs: 45.0 mi/h,
 MNM_dlink, ID: 4, type: PQ, length: 1.0 miles, ffs: 99999.0 mi/h]

In [20]:
config = dict()
config['use_link_flow'] = True
config['use_link_tt'] = False
config['link_flow_weight'] = 1
config['link_tt_weight'] = 1
config['num_data'] = 100
config['observed_links'] = [3]
config['paths_list'] = [0, 1]

In [21]:
x_real = np.random.rand(10) * 10

In [22]:
np.random.randn(10) / 10

array([-0.06828435, -0.04552275, -0.12447108,  0.21165218,  0.08223441,
        0.01864844, -0.00866736, -0.01863017, -0.04745245,  0.27119328])

In [23]:
data_dict = dict()
data_dict['link_flow'] = list()
for i in range(config['num_data']):
    link_df = pd.DataFrame(index = range(10), columns = [3], data = x_real + np.random.randn(10) / 10)
    data_dict['link_flow'].append(link_df)

In [24]:
dode = DODE(nb, config)

In [25]:
dode.add_data(data_dict)

In [ ]:
start = time.time()
dode.estimate_path_flow_mp(adagrad = True, n_process = 1)
end = time.time()
print(end - start)

In [ ]:
start = time.time()
dode.estimate_path_flow(adagrad = True)
end = time.time()
print(end - start)

[array([0, 1]), array([2, 3]), array([4])]

## SR41

In [5]:
data_folder = os.path.join('..', '..', '..', 'data', 'input_files_SR41_fix')

In [6]:
nb = MNM_network_builder()
nb.load_from_folder(data_folder)

MNM_config
MNM_pathtable


In [7]:
nb.config

[DTA]
network_name = Snap_graph
total_interval = 3000
unit_time = 5
assign_frq = 180
start_assign_interval = 0
max_interval = 6
flow_scalar = 10
num_of_link = 2065
num_of_node = 1441
num_of_O = 174
num_of_D = 174
OD_pair = 7110
routing_type = Fixed

[STAT]
rec_mode = LRn
rec_mode_para = 2
rec_folder = record
rec_volume = 0
volume_load_automatic_rec = 0
volume_record_automatic_rec = 0
rec_tt = 0
tt_load_automatic_rec = 0
tt_record_automatic_rec = 0

[FIXED]
path_file_name = path_table
num_path = 39561
choice_portion = Buffer
route_frq = 180

In [8]:
whole_observable_list = list(map(lambda x: x.ID, filter(lambda x: x.ffs < 1000, nb.link_list)))

In [9]:
obserable_list = list(filter(lambda x: np.random.rand() < 0.2, whole_observable_list))

In [10]:
paths_list = range(39561)

In [11]:
config = dict()
config['use_link_flow'] = True
config['use_link_tt'] = False
config['link_flow_weight'] = 1
config['link_tt_weight'] = 1
config['num_data'] = 4
config['observed_links'] = obserable_list
config['paths_list'] = paths_list

In [12]:
x_real = np.random.rand(nb.config.config_dict['DTA']['max_interval'], len(obserable_list) ) * 100

In [13]:
data_dict = dict()
data_dict['link_flow'] = list()
for i in range(config['num_data']):
    link_df = pd.DataFrame(index = range(nb.config.config_dict['DTA']['max_interval']), 
                       columns = obserable_list, data = x_real + 
                        np.random.randn(nb.config.config_dict['DTA']['max_interval'], len(obserable_list)) / 10)
    data_dict['link_flow'].append(link_df)

In [47]:
# data_dict = dict()
# link_df = pd.DataFrame(index = range(nb.config.config_dict['DTA']['max_interval']), 
#                        columns = obserable_list, data = x_real)
# data_dict['link_flow'] = [link_df]

In [14]:
dode = DODE(nb, config)

In [15]:
dode.add_data(data_dict)

In [30]:
start = time.time()
dode.estimate_path_flow(max_epoch = 10)
end = time.time()
print(end - start)

2
1
3
0
Epoch: 0 Loss: 7776.002312674043
2
1
3
0
Epoch: 1 Loss: 8682.001243518547
2
1
0
3
Epoch: 2 Loss: 8408.099015633667
0
1
2
3
Epoch: 3 Loss: 8001.8622263638845
3
2
1
0
Epoch: 4 Loss: 7609.498557668632
3
1
0
2
Epoch: 5 Loss: 7270.369996858141
1
0
3
2
Epoch: 6 Loss: 7454.819608290462
0
2
3
1
Epoch: 7 Loss: 7368.319316118843
0
1
3
2
Epoch: 8 Loss: 6224.911382603075
0
2
1
3
Epoch: 9 Loss: 5432.113571285203
4358.43633008


In [29]:
start = time.time()
dode.estimate_path_flow_mp(max_epoch = 10, adagrad = True, n_process = 4)
end = time.time()
print(end - start)

Epoch: 0 Loss: 2272.7760501968687
Epoch: 1 Loss: 2060.3656239354605
Epoch: 2 Loss: 1859.1707008887604
Epoch: 3 Loss: 1729.2520199387402
Epoch: 4 Loss: 1690.4164316232714
Epoch: 5 Loss: 1613.5831552144905
Epoch: 6 Loss: 1606.2572004619888
Epoch: 7 Loss: 1558.161609474343
Epoch: 8 Loss: 1562.2161446160223
Epoch: 9 Loss: 1522.7658074557448
182.438901901


In [25]:
import random
import string
# Define an output queue
output = mp.Queue()

# define a example function
def rand_string(length, output):
    """ Generates a random string of numbers, lower- and uppercase chars. """
    rand_str = ''.join(random.choice(
                        string.ascii_lowercase
                        + string.ascii_uppercase
                        + string.digits)
                   for i in range(length))
    output.put(rand_str)

# Setup a list of processes that we want to run
processes = [mp.Process(target=rand_string, args=(5, output)) for x in range(4)]

# Run processes
for p in processes:
    p.start()

    
# Get process results from the output queue
results = [output.get() for p in processes]

# Exit the completed processes
for p in processes:
    p.join()

    



print(results)

['QCPbH', 'OOK72', 'S7iwj', '0ex67']


In [ ]:
f_e = dode.init_path_flow()

In [ ]:
hash = hashlib.sha1()
hash.update(str(time.time()))
new_folder = str(hash.hexdigest())
nb.update_demand_path(f_e)
nb.dump_to_folder(new_folder)
a = MNMAPI.dta_api()
a.initialize(new_folder)
# shutil.rmtree(new_folder)
a.register_links(obserable_list)
a.register_paths(paths_list)
a.install_cc()
a.install_cc_tree()
a.run_whole()

In [1]:
nb.demand

NameError: name 'nb' is not defined

## Speed

In [43]:
dode = DODE(nb, config)

In [44]:
f_e = dode.init_path_flow()

In [45]:
dta = dode._run_simulation(f_e)

In [52]:
tt = dta.get_link_tt(np.arange(0, dode.num_loading_interval, dode.ass_freq))

In [53]:
tt.shape

(1, 10)

In [48]:
config

{'link_flow_weight': 1,
 'link_tt_weight': 1,
 'num_data': 1,
 'observed_links': [3],
 'paths_list': [0, 1],
 'use_link_flow': True,
 'use_link_tt': False}

In [49]:
tt

array([ 64.,  13.,  13.,  13.,  13.,  13.,  13.,  13.,  13.,  10.])

In [51]:
nb.get_link(3).get_fft()

64.0

In [3]:
from scipy.sparse import csr_matrix


TypeError: 'int' object is not iterable

In [5]:
csr_matrix()

<10x5 sparse matrix of type '<type 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>